In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

2022-12-27 13:11:35.625284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def prepereData():
    allData = pd.read_csv("data/songs_with_genre.csv", index_col=0,
                          header=[0, 1, 2])
    generes = allData["genre_id"]
    genres_index = pd.read_csv("data/genres_index.csv", index_col=0)
    allData.drop(columns=["genre_id"], inplace=True)
    data_x = allData
    data_x.columns = range(data_x.shape[1])
    #normalizing
    mu = np.mean(data_x, axis=0)
    sigma = (np.std(data_x, axis=0))
    data_x = (data_x - mu) / sigma
    #Creating the right lables
    generes.columns = range(generes.shape[1])
    targets = OneHotEncoder().fit_transform(genres_index).toarray()
    generes[0] = generes[0] - 1
    data_y = [targets[i - 1] for i in generes[0]]
    generes[0] = data_y
    
    test = generes.copy()
    test.rename(columns={0: "oneHotVector"}, inplace=True)
    test.shape
    split = pd.DataFrame(test["oneHotVector"].to_list(), columns=range(161))
    split.index = generes.index
    generes = split
    return data_x, generes

In [3]:
print(x_train.shape)
print(y_train.shape)
print(y_valid.shape)
print(x_valid.shape)
print(x_test.shape)
print(y_test.shape)

NameError: name 'x_train' is not defined

In [44]:
data, labels = prepereData()
#data = data.reshape(-1, 1, 518)

x_train, x_rest, y_train, y_rest = train_test_split(data, labels, train_size=0.8)
x_test, x_valid, y_test, y_valid = train_test_split(x_rest, y_rest, train_size=0.5)

In [93]:
CNN = models.Sequential()
#CNN.add(layers.InputLayer(input_shape=(x_train.shape[1],)))
CNN.add(layers.Conv1D(32, 3, activation='relu',input_shape=(518,1),padding='SAME'))
CNN.add(layers.MaxPooling1D(3))
CNN.add(layers.Conv1D(64, 4, activation='relu'))
CNN.add(layers.MaxPooling1D(4))
CNN.add(layers.Flatten())
#CNN.add(layers.Conv1D(128, 5, activation='relu'))
#CNN.add(layers.Dense(256, activation='relu'))
#CNN.add(layers.Dense(128, activation='relu'))
#CNN.add(layers.Dense(6528, activation='relu'))
CNN.add(layers.Dense(161,activation='softmax'))

In [94]:
CNN.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_32 (Conv1D)          (None, 518, 32)           128       
                                                                 
 max_pooling1d_27 (MaxPoolin  (None, 172, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_33 (Conv1D)          (None, 169, 64)           8256      
                                                                 
 max_pooling1d_28 (MaxPoolin  (None, 42, 64)           0         
 g1D)                                                            
                                                                 
 flatten_10 (Flatten)        (None, 2688)              0         
                                                                 
 dense_41 (Dense)            (None, 161)             

In [95]:
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',patience=10,verbose=1),tf.keras.callbacks.EarlyStopping(monitor='loss',mode='min',patience=10,verbose=1)]
CNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [96]:
training = CNN.fit(x_train,y_train,epochs=100, validation_data=(x_valid,y_valid),batch_size=389,callbacks=callback)

Epoch 1/100
102/102 [==============================] - 18s 172ms/step - loss: 1.6958 - accuracy: 0.4697 - val_loss: 1.3544 - val_accuracy: 0.5746
Epoch 2/100
102/102 [==============================] - 17s 166ms/step - loss: 1.2916 - accuracy: 0.5884 - val_loss: 1.2557 - val_accuracy: 0.5994
Epoch 3/100
102/102 [==============================] - 17s 165ms/step - loss: 1.2277 - accuracy: 0.6090 - val_loss: 1.2206 - val_accuracy: 0.6091
Epoch 4/100
102/102 [==============================] - 17s 162ms/step - loss: 1.1816 - accuracy: 0.6232 - val_loss: 1.1920 - val_accuracy: 0.6187
Epoch 5/100
102/102 [==============================] - 18s 176ms/step - loss: 1.1511 - accuracy: 0.6326 - val_loss: 1.1671 - val_accuracy: 0.6244
Epoch 6/100
102/102 [==============================] - 17s 163ms/step - loss: 1.1267 - accuracy: 0.6394 - val_loss: 1.1769 - val_accuracy: 0.6248
Epoch 7/100
102/102 [==============================] - 17s 170ms/step - loss: 1.1023 - accuracy: 0.6465 - val_loss: 1.1442 -

KeyboardInterrupt: 

In [72]:
x_train.shape

(39678, 518)

In [82]:
print(training.history)

{'loss': [1.178002119064331, 1.0607845783233643, 0.9766632914543152, 0.8937786221504211, 0.807569146156311, 0.7184015512466431, 0.614281952381134, 0.5079550743103027, 0.40203070640563965, 0.3137299716472626], 'accuracy': [0.6223599910736084, 0.660466730594635, 0.6820656061172485, 0.7062855958938599, 0.7317405343055725, 0.7593124508857727, 0.792630672454834, 0.8281919360160828, 0.8626694679260254, 0.8942990899085999], 'val_loss': [1.1229056119918823, 1.0694172382354736, 1.0472040176391602, 1.0581305027008057, 1.0576930046081543, 1.0892432928085327, 1.1394011974334717, 1.2091081142425537, 1.3652393817901611, 1.43087637424469], 'val_accuracy': [0.6409274339675903, 0.6540322303771973, 0.6618951559066772, 0.657056450843811, 0.6524193286895752, 0.6538306474685669, 0.6610887050628662, 0.6514112949371338, 0.6473790407180786, 0.6411290168762207]}


In [67]:
y_train

statistics,Unnamed: 519_level_1
number,Unnamed: 519_level_2
track_id,
44204,11
16191,11
27388,14
59663,1
85101,31
...,...
34204,11
33552,20


In [ ]:
test_loss, test_acc = CNN.evaluate(x_test,y_test)